# Импорты

In [21]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
###sklearn модели
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle
###sklearn метрики
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#########кодировка
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Открытие файлов, обьединение

In [22]:
data_dir = Path(r"C:\virgin_sales")
df = pd.concat([pd.read_csv(f,sep = ';', header = 1, encoding = 'cp1251') for f in data_dir.glob("*.csv")], ignore_index=True)
display(df.head(5))
df.info()

C:\Users\U\AppData\Local\Temp\ipykernel_10284\3543761218.py:2: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f,sep = ';', header = 1, encoding = 'cp1251') for f in data_dir.glob("*.csv")], ignore_index=True)


,ID,ID чека продажи,Тип чека,Округление суммы чека,Дата продажи,Товар ID,Точка продаж,Производитель,Название,Группы,...,Материал,Цвет,Аромат,"Длина, см","Диаметр, см",Размер/размер презервативов,Краткое описание МП,Рейтинг для МП,Тип презервативов,Размер бюстгальтера/Рельеф презервативов
0,271284,182757,sale,"0,00",31.01.2018 23:48:24,25642,Ленинградка,Luxe,172 Luxe MaximA Контрольный Выстрел. Презерват...,"Косметика/Презервативы,Все товары,Каталог для ...",...,NaN,NaN,Без аромата,NaN,NaN,Стандартный,NaN,97620,NaN,"С усиками,С шариками"
1,271282,182756,sale,"0,00",31.01.2018 23:05:56,29605,Ленинградка,Pjur,"96895 pjur Analyse Me Moisturising, 100мл. Ана...","Лубриканты,Все товары,Все товары,Для анального...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99946,NaN,NaN
2,271283,182756,sale,"0,00",31.01.2018 23:05:56,47449,Ленинградка,Акции,Скидка 10%,Акции,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,271201,182703,sale,"0,00",31.01.2018 23:59:55,56966,(Закрыт) Менделеевская,Прочее,открытие смены,Акции,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,271200,182702,sale,"0,00",31.01.2018 23:44:08,54674,Фрунзенская,NaN,Нетовар,Акции,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2357808 entries, 0 to 2357807
Data columns (total 38 columns):
 #   Column                                                   Dtype  
---  ------                                                   -----  
 0   ID                                                       int64  
 1   ID чека продажи                                          int64  
 2   Тип чека                                                 object 
 3   Округление суммы чека                                    object 
 4   Дата продажи                                             object 
 5   Товар ID                                                 int64  
 6   Точка продаж                                             object 
 7   Производитель                                            object 
 8   Название                                                 object 
 9   Группы                                                   object 
 10  Цена по прайслисту                        

# Удаление дубликатов

In [23]:
df=df.drop_duplicates().reset_index()

# Фоматирование и удаление лишних данных

In [24]:
df['Дата продажи'] = pd.to_datetime(df['Дата продажи'], format='%d.%m.%Y  %H:%M:%S')
df['mounth'] = pd.DatetimeIndex(df['Дата продажи']).month
df['year'] = pd.DatetimeIndex(df['Дата продажи']).year

df = df.drop(columns = ['ID', 'ID чека продажи', 'Округление суммы чека', 'Группы', 'Цена по прайслисту', 'Цена продажи',
                        'Себестоимость товара', 'Скидка', 'Характеристика', 'Срок годности проданного товара', 'ФИО продавца',
                        'Применённая ставка НДС, в процентах (на момент продажи)', 'Сумма НДС', 'Внешний ID', 'Наименование прайслиста',
                        'Бренд', 'товар или промо', 'Свойства товара для МП', 'Название товара для МП', 'Вторая часть названия товара для МП',
                        'Материал', 'Цвет', 'Аромат', 'Длина, см', 'Диаметр, см', 'Размер/размер презервативов', 'Краткое описание МП',
                        'Рейтинг для МП ', 'Тип презервативов', 'Размер бюстгальтера/Рельеф презервативов', 'index'])
def comma_replace (x):
    if x == 0:
        None
    else:
        x = str(x.replace(',','.'))
    return x

columns_to_replace = ['Количество', 'Сумма по строке']
for column in columns_to_replace:
    df[column] = df[column].apply(comma_replace)
    df[column] = df[column].astype(float)
    df[column] = df[column].astype(int)
    
df = df[df['Точка продаж'] != 'Сикретс Спб']
df = df[df['Точка продаж'] != 'Сикретс Мск']
df = df[df['Точка продаж'] != '(Закрыт) Форум Сикретс']
df = df[df['Точка продаж'] != 'Тестовая ВиртуалПос']
df = df[df['Точка продаж'] != 'Обучение Кунцевская']

os.chdir(r"C:\predict_sales")

groups = pd.read_csv('groups.csv', sep = ';', header = 0, encoding = 'cp1251')

groups = groups.drop(columns = 'Название')

groups = groups[groups['Товар ID'] != 0]


df = df.merge(groups, on ='Товар ID', how = 'left'
             )
df = df[df['Артикул'].notna()]
df['Артикул'] = df['Артикул'].astype(int)
df['Товар ID'] = df['Товар ID'].astype(int)
df = df[df['Артикул'] != 0]
df = df[df['Группа 1'] != 'Тестеры/Промо']
df = df[df['Группа 1'] != 'Прочее']
df = df[df['year'] != '2018']

# Подготовка признаков

In [25]:
pvt = pd.pivot_table(df, index=['year', 'mounth','Группа 1', 'Группа 2', 'Группа 3','ЦС']
                     , values = 'Количество', aggfunc = 'sum').reset_index()

# Деление данных на выборки

In [26]:
target = pvt['Количество']
features = pvt.drop(columns = 'Количество')



features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.15, random_state=12345,
                                                                             )

features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, 
                                                                             test_size=0.25, random_state=12345,
                                                                           )
#################encoder                              
groups_to_encode = ['Группа 1' ,'Группа 2' , 'Группа 3','ЦС']

encoders = {}

for i in groups_to_encode:
    encoders[i] = preprocessing.LabelEncoder()
    features_train[i] = encoders[i].fit_transform(features_train[i])

for i in groups_to_encode:
    features_valid[i] = encoders[i].transform(features_valid[i])

for i in groups_to_encode:
    features_test[i] = encoders[i].transform(features_test[i])


# Выбираем лучшую модель

In [28]:
# %%time
# models = ['LinearRegression','DecisionTreeRegressor','RandomForestRegressor']

# best_model = []
# best_depth = 0
# best_estimators = 0
# best_result = 100000
# state = np.random.RandomState(1)



# for model in models:
#     if model == 'DecisionTreeRegressor':
#             for depth in range(5, 26, 2):
#                 model = DecisionTreeRegressor(max_depth=depth)
#                 model.fit(features_train, target_train)
#                 predictions_valid = model.predict(features_valid)
#                 result = mean_squared_error(target_valid,predictions_valid)**0.5
#                 if result < best_result:
#                     best_model = model
#                     best_result = result
#                     best_depth = depth
#                     best_estimators = None
 
#     if model == 'RandomForestRegressor':
#         for est in range(50, 350, 10):
#             for depth in range(4, 20, 2):
#                 model = RandomForestRegressor( n_estimators = est, max_depth=depth)
#                 model.fit(features_train, target_train)
#                 predictions_valid = model.predict(features_valid)
#                 result = mean_squared_error(target_valid,predictions_valid)**0.5
#                 if result < best_result:
#                     best_model = model
#                     best_result = result
#                     best_depth = depth
#                     best_estimators = est
                    
#     if model == 'LinearRegression':
#         model = LinearRegression()
#         model.fit(features_train, target_train)
#         predictions_valid = model.predict(features_valid)
#         result = mean_squared_error(target_valid,predictions_valid)**0.5
#         if result < best_result:
#             best_model = model
#             best_result = result
#             best_depth = None
#             best_estimators = None
                    
#print(f'Лучшая модель: {best_model}, с метрикой rmse: {best_result}, и глубиной дерева: {best_depth} и estimators: {best_estimators}')

- RMSE С обьединением групп и labelencoder: 30.704997070099825 глубиной дерева: 18 и estimators: 120
- RMSE без обьединения групп и labelencoder: 32.02402317694702 глубиной дерева: 18 и estimators: 140
- Выбираем RandomForestRegressor
- Корректируем обучающие данные и ищем наиболее точный вариант

In [29]:
%%time
best_depth = 0
best_estimators = 0
best_result = 10000

# model = RandomForestRegressor(n_estimators = 210, max_depth=22) # Лучшая MAE модель
# model.fit(features_train, target_train)
# predictions_valid = model.predict(features_valid)
# result = mean_absolute_error(target_valid,predictions_valid)

model = RandomForestRegressor(random_state =12345, n_estimators = 170, max_depth=24) # Лучшая RMSE модель
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = mean_absolute_error(target_valid,predictions_valid)

# model = RandomForestRegressor(n_estimators = 250, max_depth=20) # Лучшая R2 модель
# model.fit(features_train, target_train)
# predictions_valid = model.predict(features_valid)
# result = mean_absolute_error(target_valid,predictions_valid)


# for est in range(10, 300, 10):
#     for depth in range(18, 27, 1):
#         model = RandomForestRegressor(n_estimators = est, max_depth=depth)
#         model.fit(features_train, target_train)
#         predictions_valid = model.predict(features_valid)
#         result = mean_squared_error(target_valid,predictions_valid)**0.5
#         if result < best_result:
#             best_model = model
#             best_result = result
#             best_depth = depth
#             best_estimators = est
            
            
            

#print(f'Метрика rmse: {best_result}, и глубина дерева: {best_depth} и estimators: {best_estimators}')


Wall time: 2.06 s


- лучшая модель MAE: глубина дерева: 22 и estimators: 210
- лучшая модель RMSE: глубина дерева: 24  и estimators: 170
- лучшая модель R2: глубина дерева: 20 и estimators: 250

In [30]:
print("R2 =", r2_score(target_valid, predictions_valid))
print("MAE =", mean_absolute_error(target_valid, predictions_valid))
print("RMSE =", mean_squared_error(target_valid, predictions_valid)**0.5)

R2 = 0.9629143988797242
MAE = 12.179775557653194
RMSE = 25.143732278403657


1. RMSE без обьединения групп и labelencoder со статистикой 2018 года: 32.02402317694702 глубина дерева: 18 и estimators: 140
2. RMSE без обьединения групп и labelencoder и без статистики 2018 года: 29.540001954561905 глубина дерева: 18 и estimators: 210
3. RMSE без обьединения групп и дамми - переменными и без статистики 2018 года: 40.45008557723137
4. RMSE без обьединения групп и binaryencoder и без статистики 2018 года: 125.00716078514348


6. Показатель лучшей модели по метрике MAE: 
- MAE  = 13.62635162606571
- R2 = 0.9474480883201776
- RMSE = 29.305230932433016
7. Показатель лучшей модели по метрике RMSE:
- MAE = 13.724088612897408
- R2 = 0.9466833268174901
- RMSE = 28.803204900106405
8. Показатель лучшей модели по метрике R2: 
- MAE = 13.76371919532531
- R2 = 0.9465612011251503
- RMSE = 29.551479761783142


# Предсказания на тестовой выборке

In [31]:
predictions_test = model.predict(features_test)
result = mean_squared_error(target_test,predictions_test)**0.5


# Декодирование категориальных переменных

In [32]:
for i in groups_to_encode:
    features_test[i] = encoders[i].inverse_transform(features_test[i])


In [33]:
features_test = features_test.reset_index(drop = True)
target_test = target_test.reset_index(drop = True)

features_test['actual'] = target_test
predictions_test = round(pd.Series((predictions_test))).astype(int)
features_test['predicted'] = predictions_test

# Прогнозирование продаж

In [34]:
features_all = pd.read_csv('features_to_predict.csv',sep = ';', header = 0, encoding = 'cp1251')
##################################################################

year = 2023 # Прогнозируемый год
mounths = [2,3,4] #Номера месяцев прогнозирумого квартала

##################################################################
features_to_predict = []

for index, line in features_all.iterrows():
    for mounth in mounths:
        features_to_predict.append(
            (year, mounth,
                                   line['Группа 1'], line['Группа 2'], line['Группа 3'],
                                  line['ЦС'])
        )
        
features_to_predict = pd.DataFrame(features_to_predict, columns = 
                                   ['year', 'mounth', 'Группа 1', 'Группа 2', 'Группа 3', 'ЦС'])        

for i in groups_to_encode:
    features_to_predict[i] = encoders[i].transform(features_to_predict[i])

predictions = model.predict(features_to_predict)

for i in groups_to_encode:
    features_to_predict[i] = encoders[i].inverse_transform(features_to_predict[i])
    
def glue (row):
    group_1 = row['Группа 1']
    group_2 = row['Группа 2']
    group_3 = row['Группа 3']
    price_group = row['ЦС']
    return group_1+group_2+group_3+price_group

features_to_predict['glued_groups'] = features_to_predict.apply(glue, axis = 1)
predictions = pd.Series(predictions)
features_to_predict['predict'] = predictions

In [35]:
display(features_to_predict[features_to_predict['Группа 3'] == 'С рельефом'])

display(features_to_predict['ЦС'].unique())

,year,mounth,Группа 1,Группа 2,Группа 3,ЦС,glued_groups,predict
861,2023,2,Презервативы,Латекс,С рельефом,A,ПрезервативыЛатексС рельефомA,320.705882
862,2023,3,Презервативы,Латекс,С рельефом,A,ПрезервативыЛатексС рельефомA,293.517647
863,2023,4,Презервативы,Латекс,С рельефом,A,ПрезервативыЛатексС рельефомA,331.794118
864,2023,2,Презервативы,Латекс,С рельефом,B,ПрезервативыЛатексС рельефомB,258.664706
865,2023,3,Презервативы,Латекс,С рельефом,B,ПрезервативыЛатексС рельефомB,214.582353
866,2023,4,Презервативы,Латекс,С рельефом,B,ПрезервативыЛатексС рельефомB,185.988235
900,2023,2,Презервативы,Латекс,С рельефом,C,ПрезервативыЛатексС рельефомC,422.452941
901,2023,3,Презервативы,Латекс,С рельефом,C,ПрезервативыЛатексС рельефомC,311.394118
902,2023,4,Презервативы,Латекс,С рельефом,C,ПрезервативыЛатексС рельефомC,229.582353
1227,2023,2,Презервативы,Латекс,С рельефом,A+,ПрезервативыЛатексС рельефомA+,98.729412


array(['A', 'C', 'B', 'A+'], dtype=object)

# Распределение прогноза внутри группы

In [36]:
array = pd.read_csv('matriza.csv', sep = ';', header = 0, encoding = 'cp1251')
array = array[array['Black'] != 'black' ]
array = array[array['Среднемес. продажа по сети'] != '-']

array['glued_groups'] = array.apply(glue, axis = 1)
array['Среднемес. продажа по сети'] = array['Среднемес. продажа по сети'].apply(comma_replace)
array['Среднемес. продажа по сети'] = array['Среднемес. продажа по сети'].astype(float)


def group_ratio (row):
    glued = row['glued_groups']
    sales_rows = array[array['glued_groups'] == glued]
    sales = sales_rows['Среднемес. продажа по сети'].sum()
    item_sales = row['Среднемес. продажа по сети']
    ratio = item_sales/sales
    return ratio

array['ratio'] = array.apply(group_ratio, axis = 1)

array = array.merge(features_to_predict[['glued_groups','predict']],
                    how = 'left',on = 'glued_groups')

array['to_order'] = (
    round(
        array['ratio']*array['predict'])
    .fillna(0)
    .astype(int)
)

array_final = (pd.pivot_table(array,
                            values ='to_order',
                            index = ['Артикул', 'Название','Группа 1','Группа 2','Группа 3','ЦС'],
                            aggfunc = sum)
                            .reset_index())

array_final.to_excel(f'Закупка {mounths}.xlsx', index = False)

In [37]:
display(array[array['Группа 3'] == 'С рельефом'])

,Товар ID,Артикул,Название,Бренд,Товар промо,Цена :: Балансовая стоимость,Цена :: Цена обычного магазина,Группа 1,Группа 2,Группа 3,ЦС,Среднемес. продажа по сети,Рейтинг группы 3,Black,glued_groups,ratio,predict,to_order
93,25638,168,168 Luxe MaximA Глубинная Бомба. Презервативы ...,Luxe,товар,97.0,199.0,Презервативы,Латекс,С рельефом,C,44.0,1.0,NaN,ПрезервативыЛатексС рельефомC,0.086162,422.452941,36
94,25638,168,168 Luxe MaximA Глубинная Бомба. Презервативы ...,Luxe,товар,97.0,199.0,Презервативы,Латекс,С рельефом,C,44.0,1.0,NaN,ПрезервативыЛатексС рельефомC,0.086162,311.394118,27
95,25638,168,168 Luxe MaximA Глубинная Бомба. Презервативы ...,Luxe,товар,97.0,199.0,Презервативы,Латекс,С рельефом,C,44.0,1.0,NaN,ПрезервативыЛатексС рельефомC,0.086162,229.582353,20
96,74328,45529,"45529 Maxus Special, 3 шт. Презервативы в желе...",Maxus,товар,153.0,599.0,Презервативы,Латекс,С рельефом,B,127.0,1.0,NaN,ПрезервативыЛатексС рельефомB,0.787190,258.664706,204
97,74328,45529,"45529 Maxus Special, 3 шт. Презервативы в желе...",Maxus,товар,153.0,599.0,Презервативы,Латекс,С рельефом,B,127.0,1.0,NaN,ПрезервативыЛатексС рельефомB,0.787190,214.582353,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3246,25642,172,172 Luxe MaximA Контрольный Выстрел. Презерват...,Luxe,товар,97.0,199.0,Презервативы,Латекс,С рельефом,C,12.0,18.0,NaN,ПрезервативыЛатексС рельефомC,0.023499,311.394118,7
3247,25642,172,172 Luxe MaximA Контрольный Выстрел. Презерват...,Luxe,товар,97.0,199.0,Презервативы,Латекс,С рельефом,C,12.0,18.0,NaN,ПрезервативыЛатексС рельефомC,0.023499,229.582353,5
3341,25651,175,175 Luxe ExclusivE Ночной разведчик. Презерват...,Luxe,товар,92.0,199.0,Презервативы,Латекс,С рельефом,C,12.0,19.0,NaN,ПрезервативыЛатексС рельефомC,0.023499,422.452941,10
3342,25651,175,175 Luxe ExclusivE Ночной разведчик. Презерват...,Luxe,товар,92.0,199.0,Презервативы,Латекс,С рельефом,C,12.0,19.0,NaN,ПрезервативыЛатексС рельефомC,0.023499,311.394118,7
